# phylogeny of global M. bovis lineages

Make phylogeny from selected samples and delineate lineages

* global https://www.frontiersin.org/articles/10.3389/fmicb.2020.00843/full
* UK https://www.ncbi.nlm.nih.gov/bioproject/PRJNA523164

In [1]:
import sys,os,shutil,subprocess
import glob,random
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
import pylab as plt
import matplotlib as mpl
import seaborn as sns
from Bio import SeqIO
sys.path.append('pathogenie')
from snipgenie import tools, aligners, app, trees, plotting
import toytree

In [9]:
#samples = pd.read_csv('/storage/btbgenie/global_results/samples.csv')
#app.clean_bam_files(samples,'/storage/btbgenie/global_results/mapped/',True)

In [ ]:
reload(app)
reload(tools)
reload(aligners)
args = {'threads':16, 'outdir': '/storage/btbgenie/global_results', 'labelsep':'-',
        'input':['/storage/btbgenie/brites'],
         'species': 'Mbovis-AF212297', 'overwrite':False,
         'custom_filters': True,
         'buildtree':True}
W = app.WorkFlow(**args)
st = W.setup()
W.run()

In [ ]:
treefile1 = '/storage/btbgenie/global_results/tree.newick'
tre = toytree.tree(treefile1)
mystyle = { "layout": 'c','node_sizes':1,'tip_labels_align':True}
tre.draw(**mystyle,height=700);

## spoligotypes

In [ ]:
def get_spoligotypes(samples, spo=None):
    """Get spoligotypes for multiple M.bovis strains"""
    
    if spo is not None:
        done=list(spo['sample'])
    else:
        done=[]
    samples = samples.drop_duplicates('sample')
    res=[]
    for i,r in samples.iterrows():       
        f=r.filename1
        samp=r['sample']
        if samp in done:
            continue
        b = tools.get_spoligotype(f)
        sb = tools.get_sb_number(b)
        print (r['sample'], sb, b)
        res.append([r['sample'],sb,b])
    
    res = pd.DataFrame(res,columns=['sample','SB','code'])
    return res

spo = pd.read_csv('/storage/btbgenie/global_results/spotypes.csv')
samples = pd.read_csv('/storage/btbgenie/global_results/samples.csv')
res = get_spoligotypes(samples,spo)
#res=pd.concat([spo,res])
#res.to_csv('/storage/btbgenie/global_results/spotypes.csv')

## RD analysis

In [ ]:
samples = pd.read_csv('/storage/btbgenie/global_results/samples.csv')
from snipgenie import rdiff
reload(rdiff)
rdiff.create_rd_index()
res = rdiff.run_samples(samples,'/storage/btbgenie/global_results/rd_aligned',threads=10)
X = rdiff.get_matrix(res, cutoff=0.15)
X['ident'] = X.apply(rdiff.apply_rules,1)
X.to_csv('/storage/btbgenie/global_results/rd.csv')

In [ ]:
reload(rdiff)
rdiff.show_rd_coverage(samples[:5], 'LT708304.1',4330000,4340000,app.mbovis_genome,margin=1300)

## merge to get metadata 

In [39]:
samples = pd.read_csv('/storage/btbgenie/global_results/samples.csv')
meta = pd.read_csv('brites_table1.csv')
x = samples.merge(meta,right_on='Accession_Number',left_on='sample',how='left')
spotypes = pd.read_csv('/storage/btbgenie/global_results/spotypes.csv')
R = pd.read_csv('/storage/btbgenie/global_results/rd.csv',index_col=0)
x = x.merge(spotypes,on='sample')
rds=R[['RD1mic','RD149','RD3','RD5','RD5oryx*','RD122','RDAf2','RDWicklow']]
meta = x.merge(rds,left_on='sample',right_index=True)
len(meta)

211

In [42]:
complexcolors = {'La2':'blue', 'La1.8.1':'pink', 'La1.3':'', 'La1.6':'', 'La1.7.1':'', 'La1.5':'', 'La3':'',
                'La1.8.2':'', 'La1.7-unk4':'', 'La1.2':'', 'La1.4':'', 'La1.7-unk5':'', 'La1.1':''}


array(['La2', 'La1.8.1', 'La1.3', 'La1.6', 'La1.7.1', 'La1.5', 'La3',
       'La1.8.2', 'La1.7-unk4', 'La1.2', 'La1.4', 'La1.7-unk5', 'La1.1'],
      dtype=object)

In [64]:
R = pd.read_csv('/storage/btbgenie/global_results/rd.csv',index_col=0)
#mapdict=dict(zip(meta['sample'],meta['SIB_Number']))
#X.index=X.index.map(mapdict)
scols=['sample','Clonal_Complex']
R=R.merge(meta[scols],left_index=True,right_on='sample',how='left').set_index('sample')

row_colors,colormap = get_color_mapping(R,'Clonal_Complex',cmap='Paired',seed=18)

cols = ['RD1','RD1mic','RDWicklow','RD8','RD11','RD149','RD6','RD3','RD122','RDAf2']


import matplotlib.patches as mpatches
g=sns.clustermap(R[cols],lw=0,cmap='gray_r',fmt='g',cbar_pos=None,col_cluster=False,
                 yticklabels=False,row_colors=row_colors,figsize=(12,12))
pts=[]
for c in colormap:
    pts.append(mpatches.Patch(color=colormap[c],label=c))
g.fig.legend(handles=pts,bbox_to_anchor=(1.15, .9))

In [ ]:
#treefile1 = '/storage/btbgenie/global_results/tree.newick'
#clusts = trees.get_clusters(treefile1)
#meta = meta.merge(clusts,left_on='sample',right_on='SequenceName')

In [41]:
meta.to_csv('/storage/btbgenie/global_results/metadata.csv',index=False)

## combine ireland + global samples

add Ireland samples to global tree

In [43]:
outdir='/storage/btbgenie/combined_results'

folders = {'wicklow':'/storage/btbgenie/wicklow_results',
          'other':'/storage/btbgenie/misc_results/',
          'monaghan':'/storage/btbgenie/monaghan/monaghan_results',
          'NI':'/storage/btbgenie/NI_results/',
          'NI_AFBI':'/storage/btbgenie/NI_AFBI_results/',
          'global':'/storage/btbgenie/global_results/'}

x=[]
for f in folders:
    print (f)
    file = os.path.join(folders[f],'samples.csv')   
    df=pd.read_csv(file)
    if f != 'global':
       df = df.sample(25,random_state=1) 
    x.append(df)
allsamples = pd.concat(x)
print (len(allsamples))
allsamples.to_csv(os.path.join(outdir,'samples.csv'),index=False)
app.write_samples(allsamples[['sample']], outdir)

wicklow
other
monaghan
NI
NI_AFBI
global
336


'/storage/btbgenie/combined_results/samples.txt'

## run variant calling on combined files

In [ ]:
reload(app)
bam_files = allsamples.bam_file
gff_file=os.path.join(outdir,'mbovis.gff')
tools.gff_bcftools_format(app.mbovis_gb, gff_file)
app.run_bamfiles(bam_files, app.mbovis_genome, outdir='/storage/btbgenie/combined_results', threads=16,
                 mask=app.mbovis_mask, gff_file=gff_file,
                 custom_filters=True, overwrite=False)

<div class="alert alert-block alert-info"><b>Start here for post analysis</b></div>

In [52]:
scols=['sample','SB','batch','county']
ireland = pd.read_csv('/storage/btbgenie/all_ireland_results/metadata.csv')
ireland['Country']='Ireland'
print (len(ireland), len(meta))
master=pd.concat([ireland,meta])

567 211


In [53]:
def get_common(x, common):
    if x in common:
        return x
    else:
        return 'Other'

In [54]:
common = master.Country.value_counts()[:9]
master['Country1'] = master.Country.apply(lambda x: get_common(x, common))
len(master)

778

In [55]:
common = master.county.value_counts()[:4]
master['county1'] = master.county.apply(lambda x: get_common(x, common))
master['cc'] = master['Clonal_Complex'].fillna(master.Country)
master['cc1'] = master['Clonal_Complex'].fillna(master.county1)

In [56]:
common = master.SB.value_counts()[:8]
master['SB1'] = master.SB.apply(lambda x: get_common(x, common))

TreeCluster.py  -i /storage/btbgenie/combined_results/tree.newick -t 3 -m max_clade
TreeCluster.py  -i /storage/btbgenie/combined_results/tree.newick -t 5 -m max_clade
TreeCluster.py  -i /storage/btbgenie/combined_results/tree.newick -t 7 -m max_clade
TreeCluster.py  -i /storage/btbgenie/combined_results/tree.newick -t 10 -m max_clade
TreeCluster.py  -i /storage/btbgenie/combined_results/tree.newick -t 12 -m max_clade
TreeCluster.py  -i /storage/btbgenie/combined_results/tree.newick -t 20 -m max_clade
TreeCluster.py  -i /storage/btbgenie/combined_results/tree.newick -t 50 -m max_clade
TreeCluster.py  -i /storage/btbgenie/combined_results/tree.newick -t 100 -m max_clade
[11, 14, 34, 5, 9, 1, 40, 3, 29, 24, 15, 19, 35, 44, 30, 6, 8, 18, 17]


In [57]:
treefile = '/storage/btbgenie/combined_results/tree.newick'
clusts = trees.get_clusters(treefile)
cc = clusts.snp100.value_counts()[1:20]
cc.plot(kind='bar',figsize=(12,5))
print (list(cc.index))

In [231]:
#master = master.merge(clusts,left_on='sample',right_on='SequenceName')

In [58]:
master.to_csv('/storage/btbgenie/combined_results/metadata.csv')
len(master)

778

In [ ]:
#tre = tre.root(names=['SRR5486072'])
def toytree_draw(tre, meta, labelcol,colorcol):
    """Draw colored tree with toytree"""
    
    tipnames = tre.get_tip_labels()
    mapping = dict(zip(meta['sample'],meta[labelcol]))
    mapping['ref'] = 'AF2122/97'
    tiplabels = [mapping[i] if i in mapping else 'NA' for i in tipnames]   
    mapping = dict(zip(meta['sample'],meta[colorcol]))
    colormap = trees.colors_from_labels(meta,'sample',colorcol)
    tip_colors = [colormap[mapping[i]] if i in mapping else 'Black' for i in tipnames]
    node_sizes=[0 if i else 8 for i in tre.get_node_values(None, 1, 0)]
    node_colors = [colormap[mapping[n]] if n in mapping else 'black' for n in tre.get_node_values('name', True, True)]

    canvas,t,r=tre.draw(layout='r',width=600,height=800,tip_labels=tiplabels,node_markers="o",node_hover=True,edge_widths=1,
             tip_labels_colors=tip_colors,node_sizes=node_sizes,scalebar=True,node_colors=node_colors)#tip_labels_align=True);
    #canvas.legend(leg,corner=("top", 100, 100, 70));
    return

#import toyplot
#toyplot.png.render(canvas, "/storage/btbgenie/global_results/tree_clusters.png")
toytree_draw(tre2, master,'Country1','SB')

In [ ]:
#combine rd data
rd1=pd.read_csv('/storage/btbgenie/global_results/rd.csv',index_col=0)
rd2=pd.read_csv('/storage/btbgenie/mbovis_ireland/rd.csv',index_col=0)
rd=pd.concat([rd1,rd2])

In [251]:
cols=['sample','SB']
rvar = ['RD149','RD3','RD5','RD5oryx*','RD122','RDAf2','RDWicklow']
X = rd[rvar]
X=X.merge(master[cols],left_index=True,right_on='sample').set_index(cols)
#X = X.iloc[:,1:]
#print (X)
sns.clustermap(X,xticklabels=True,yticklabels=[],cmap='gray',figsize=(12,12))

In [61]:
print(plotting.random_colors(12))

['#442082', '#3CFDE6', '#F1C26B', '#30F90E', '#C7DD01', '#E48875', '#34A20F', '#0B0D04', '#C36ED8', '#0E71E0', '#FD77B0', '#7670EB']


In [ ]:
names = tre2.get_tip_labels()
names.reverse()
#print (names)
rdv=rd[rvar]
matrix = rdv.loc[names].values*255

In [ ]:
# create a canvas
import toyplot
canvas = toyplot.Canvas(width=600, height=650);

# add tree
axes = canvas.cartesian(bounds=(50, 350, 20, 640))
tre2.draw(axes=axes,tip_labels_colors=tip_colors,#node_colors=node_colors, #,tip_labels=tiplabels #
         tip_labels_align=True)

# add matrix
table = canvas.table(
    rows=len(rdv),
    columns=6,
    margin=0,
    bounds=(375, 450, 25, 630),  
)

colormap = toyplot.color.brewer.map("Blues")

# apply a color to each cell in the table
for ridx in range(matrix.shape[0]):
    for cidx in range(matrix.shape[1]):
        cell = table.cells.cell[ridx, cidx]
        cell.style = {
            "fill": colormap.colors(matrix[ridx, cidx], 0, 100),
        }

# style the gaps between cells
table.body.gaps.columns[:] = 3
table.body.gaps.rows[:] = 3


## SNPs and indels clustering

In [ ]:
csq = app.read_csq_file('/storage/btbgenie/combined_results/csq.tsv')
snp_mat = app.get_aa_snp_matrix(csq)

In [ ]:
csqind = app.read_csq_file('/storage/btbgenie/combined_results/csq_indels.tsv')
#print (csqind)
#print (csqind.snp_type.value_counts())

ind_mat = app.get_aa_snp_matrix(csqind)
ind_mat.reset_index().snp_type.value_counts()
#ind_mat
#ind_mat[ind_mat.sum(1)>5]

In [ ]:
mat = snp_mat
#mat.index = mat.index.get_level_values(1)+'_'+mat.index.get_level_values(2)
stat = mat.sum(1).reset_index()
#stat.gene.value_counts()[:30]

In [ ]:
C=mat.T
C=C.merge(meta[cols],left_index=True,right_on='SRA ID').set_index(cols).T
C=C[C.sum(1)>3].T
C.iloc[:4,:2]

In [ ]:
countries = C.index.get_level_values(1)
sb = C.index.get_level_values(2)
clust = C.index.get_level_values(3)

In [ ]:
reload(trees)
colormap1 = dict(zip(sb.unique(), trees.qcolors))
colors1 = [colormap1[i] if i in colormap1 else 'Black' for i in sb]

colormap2 = dict(zip(clust.unique(), trees.qcolors))
colors2 = [colormap2[i] if i in colormap2 else 'Black' for i in clust]
#print (colors2)

g=sns.clustermap(C,xticklabels=False,yticklabels=True,figsize=(17,12),cmap='gray_r',cbar_pos=None,row_colors=colors2)
g.fig.savefig('/storage/btbgenie/global_results/indel_clusters.jpg')

## PCA snps

In [ ]:
import sklearn
from sklearn import decomposition
from sklearn import manifold

pca = decomposition.PCA(n_components=3)
mds = manifold.MDS(n_components=3)
pos = mds.fit(C).embedding_
X = pca.fit_transform(pos)
#X = pca.transform(C)

fig,ax = plt.subplots(figsize=(7, 7))
ax.scatter(X[:, 0], X[:, 1], c=colors2, s=100, alpha=.6)
